**Title:** DistilBERT with Dynamic Text Cleaning using LLM

**Summary:**
This notebook extends the functionality of a DistilBERT model for text classification by incorporating dynamic text cleaning using Language Model Fine-Tuning. The primary goal is to enhance the robustness and effectiveness of the classification model by dynamically cleaning potentially offensive or harmful text inputs before prediction.

**Key Features:**
1. **DistilBERT Model for Text Classification:** The notebook utilizes a DistilBERT (Distilled BERT) model for sequence classification. DistilBERT is a lightweight version of BERT that offers faster inference without compromising performance significantly.

2. **Dynamic Text Cleaning with Language Model Fine-Tuning:** Similar to BERT, the notebook integrates OpenAI's Language Model Fine-Tuning (LLM) to dynamically clean potentially offensive or harmful text inputs before passing them to the DistilBERT model for classification. This ensures that the model receives sanitized inputs, improving its performance and reliability.

3. **Real-Time Text Classification:** The implemented script allows users to interactively input text for classification. The integrated text cleaning ensures that even if the input contains offensive language or hate speech, the model provides predictions based on non-offensive versions of the input text.

4. **Enhanced Speech Processing and Prediction:** The notebook demonstrates an iterative approach to text processing and prediction, where potentially offensive inputs are automatically sanitized before classification. This enhances the usability and safety of the model for real-world applications.

**Usage:**
- Users can leverage this notebook to build and deploy text classification models with enhanced robustness against offensive or harmful content.
- The integrated real-time text classification script allows for on-the-fly analysis of text inputs, making it suitable for applications requiring live content moderation or analysis.
- The combination of DistilBERT for classification and LLM for dynamic text cleaning provides a comprehensive solution for processing user-generated content in various applications, including social media monitoring, online forums moderation, and content filtering.

In [12]:
import re
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [13]:
emoticons = [':-)', ':)', '(:', '(-:', ':))', '((:', ':-D', ':D', 'X-D', 'XD', 'xD', 'xD', '<3', '3', ':*', ':-*', 'xP', 'XP', 'XP', 'Xp', ':-|', ':->', ':-<', '8-)', ':-P', ':-p', '=P', '=p', ':*)', '*-*', 'B-)', 'O.o', 'X-(', ')-X']

def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://[^\s]+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'\d+', '', text)
    for emoticon in emoticons:
        text = text.replace(emoticon, '')
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text = re.sub(r"([?.!,¿])", r" ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text.strip()


In [14]:
# Load dataset
df = pd.read_csv('/kaggle/input/dataset/labeled_data.csv')
df['tweet'] = df['tweet'].apply(clean_text)

# Split dataset
train_texts, temp_texts, train_labels, temp_labels = train_test_split(df['tweet'], df['class'], test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)


In [15]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)


In [16]:
# Dataset class
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels.tolist())
test_dataset = TweetDataset(test_encodings, test_labels.tolist())
val_dataset = TweetDataset(val_encodings, val_labels.tolist())

In [17]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [18]:
# Model initialization
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)  # Adjust num_labels according to your classification problem
optimizer = AdamW(model.parameters(), lr=5e-6)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training function
def train(epoch):
    model.train()
    total_loss, total_accuracy = 0, 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        logits = outputs.logits.detach().cpu().numpy()
        predictions = np.argmax(logits, axis=-1)
        total_accuracy += accuracy_score(labels.cpu().numpy(), predictions)
    
    avg_loss = total_loss / len(train_loader)
    avg_accuracy = total_accuracy / len(train_loader)
    print(f"Training Loss: {avg_loss:.3f}")
    print(f"Training Accuracy: {avg_accuracy:.3f}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
# Evaluation function
def evaluate(loader, desc="Evaluating"):
    model.eval()
    total_loss, total_accuracy = 0, 0
    all_predictions, all_labels = [], []
    
    for batch in tqdm(loader, desc=desc):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss.item()
        total_loss += loss
        logits = outputs.logits.detach().cpu().numpy()
        predictions = np.argmax(logits, axis=-1)
        total_accuracy += accuracy_score(labels.cpu().numpy(), predictions)
        
        all_predictions.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    avg_accuracy = total_accuracy / len(loader)
    print(f"Validation Loss: {avg_loss:.3f}")
    print(f"Validation Accuracy: {avg_accuracy:.3f}")
    
    return all_labels, all_predictions

In [21]:
# Main training loop
for epoch in range(1, 4):
    train(epoch)
    evaluate(val_loader)

# Final evaluation on test set
labels, predictions = evaluate(test_loader, "Final Test Evaluation")
print(classification_report(labels, predictions, target_names=['Hate Speech', 'Offensive Language', 'Neither']))

# Accuracy
accuracy = accuracy_score(labels, predictions)
print(f"Test Accuracy: {accuracy:.3f}")

Training Epoch 1: 100%|██████████| 543/543 [00:47<00:00, 11.49it/s]


Training Loss: 0.408
Training Accuracy: 0.860


Evaluating: 100%|██████████| 117/117 [00:03<00:00, 35.95it/s]


Validation Loss: 0.291
Validation Accuracy: 0.899


Training Epoch 2: 100%|██████████| 543/543 [00:47<00:00, 11.50it/s]


...

Training Loss: 0.221
Training Accuracy: 0.923


Evaluating: 100%|██████████| 117/117 [00:03<00:00, 36.08it/s]


Validation Loss: 0.245
Validation Accuracy: 0.915


Final Test Evaluation: 100%|██████████| 117/117 [00:03<00:00, 36.93it/s]

Validation Loss: 0.233
Validation Accuracy: 0.914
                    precision    recall  f1-score   support

       Hate Speech       0.51      0.23      0.32       207
Offensive Language       0.93      0.96      0.95      2880
           Neither       0.88      0.91      0.89       631

          accuracy                           0.91      3718
         macro avg       0.77      0.70      0.72      3718
      weighted avg       0.90      0.91      0.90      3718

Test Accuracy: 0.913


In [22]:
import pandas as pd

# Load the dataset (replace 'path_to_your_dataset.csv' with your actual dataset path)
df = pd.read_csv('/kaggle/input/dataset/labeled_data.csv')

# Filter the dataset for hate speech comments
hate_speech_comments = df[df['class'] == 0]

# Display the hate speech comments
print("Number of hate speech comments:", hate_speech_comments.shape[0])
print("Examples of hate speech comments:")
print(hate_speech_comments[['tweet']].head())  # Display the first few comments


Number of hate speech comments: 1430
Examples of hate speech comments:
                                                 tweet
85   "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...
89   "@CB_Baby24: @white_thunduh alsarabsss" hes a ...
110  "@DevilGrimz: @VigxRArts you're fucking gay, b...
184  "@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL...
202  "@NoChillPaz: "At least I'm not a nigger" http...


## Real-Time Text Classification Script

This Python script is designed to classify text inputs in real-time, making it an invaluable tool for monitoring and analyzing user-generated content live. It can identify if the text is hate speech, offensive language, or neither.

### Script Overview

The script engages with the user in an interactive session where it continuously accepts text inputs. Each input is processed to determine its classification based on predefined categories: Hate Speech, Offensive Language, or Neither. This is particularly useful for applications that require live moderation or instant text analysis.

### Code Functionality

- **Text Cleaning**: Initially, the text provided by the user is cleaned to remove any unwanted characters or formatting.
- **Text Tokenization and Encoding**: The cleaned text is tokenized and encoded using a pre-configured tokenizer and model setup.
- **Model Prediction**: The tokenized text is fed into a neural network model, which evaluates the text and produces a prediction.
- **Classification**: The output from the model is interpreted as one of the three categories based on the highest probability.
- **Confidence Scores**: Alongside the classification, the script also outputs the confidence scores for each category, providing insight into the model's decision-making process.


In [23]:
def preprocess_and_predict(text):
    # Clean the text
    cleaned_text = clean_text(text)

    # Tokenize the text
    encodings = tokenizer(cleaned_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Move tensors to the same device as model
    encodings = {key: val.to(device) for key, val in encodings.items()}

    # Evaluation mode
    model.eval()

    # Forward pass, no need to compute gradients
    with torch.no_grad():
        outputs = model(**encodings)
    
    # Get predictions
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predictions = torch.argmax(probabilities, dim=-1)

    # Convert predictions to labels
    label_map = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
    predicted_label = label_map[predictions.item()]

    # Get confidence scores
    confidence_scores = probabilities.squeeze().tolist()  # convert to list of probabilities

    return predicted_label, confidence_scores

def main():
    while True:
        user_input = input("Enter a tweet to analyze (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        predicted_label, confidence_scores = preprocess_and_predict(user_input)
        print("Predicted label:", predicted_label)
        print("Confidence Scores:", confidence_scores)

# Run the main function
if __name__ == "__main__":
    main()


Enter a tweet to analyze (or type 'exit' to quit):  I really do hate you


Predicted label: Neither
Confidence Scores: [0.03413306921720505, 0.135234072804451, 0.8306328654289246]


Enter a tweet to analyze (or type 'exit' to quit):  you are worthless idiot person


Predicted label: Offensive Language
Confidence Scores: [0.3015950620174408, 0.5868861675262451, 0.11151880770921707]


Enter a tweet to analyze (or type 'exit' to quit):  I dont like you


Predicted label: Neither
Confidence Scores: [0.049357328563928604, 0.1588636189699173, 0.7917789816856384]


Enter a tweet to analyze (or type 'exit' to quit):  exit


## Text Cleaning Function

This Python script uses the OpenAI API to transform potentially offensive or hate speech into polite and non-offensive language. It leverages the power of OpenAI's advanced natural language processing models to interpret and reformulate the input text.

### Functionality

- The script defines a function `clean_speech` that takes a string input.
- It sends this input to the OpenAI API, requesting a rewritten version that is non-offensive.
- The API's response is processed and the "cleaned" text is returned.

### Usage

1. **Input**: The user is prompted to enter a sentence that may contain offensive or hate speech.
2. **Processing**: The entered text is sent to the OpenAI API, which processes the text and generates a non-offensive version.
3. **Output**: The cleaned text is printed out.

### Setup

To run this script, you will need:
- Python installed on your machine.
- The `openai` Python library installed. You can install it using pip:
  ```bash
  pip install openai


In [ ]:
import openai

def clean_speech(input_text):
    try:
        # Use the OpenAI API to "translate" offensive speech to non-offensive speech
        response = openai.Completion.create(
          engine="text-davinci-003",  # You can choose a suitable model like davinci or curie
          prompt=f"Rewrite the following sentence to be polite and non-offensive:\n\n{input_text}",
          max_tokens=60  # Adjust max_tokens as necessary
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return str(e)

# Receive input from the user
user_input = input("Enter a potentially offensive sentence: ")
cleaned_text = clean_speech(user_input)
print("Cleaned Text:", cleaned_text)


## Enhanced Text Analysis and Cleaning Function

This updated Python script adds an integration with the OpenAI API to an existing text processing pipeline. The script classifies input text as Hate Speech, Offensive Language, or Neither and uses OpenAI's API to convert offensive content into non-offensive language.

### Functionality

- **Text Preprocessing and Prediction**: The input text is cleaned, tokenized, and passed through a machine learning model to classify its nature.
- **Integration with OpenAI API**: If the text is classified as Hate Speech or Offensive Language, it is then sent to the OpenAI API, which rewrites it to be polite and non-offensive.
- **Output**: The script outputs the classification label, confidence scores for each class, and, if applicable, the cleaned text.

### Workflow

1. **Input**: Continuously takes user input until 'exit' is entered.
2. **Classification and Cleaning**: Depending on the text's classification:
   - If offensive, it is cleaned using OpenAI.
   - If not offensive, the original text classification and confidence scores are output.
3. **Display**: Outputs the results including any cleaned text.

### Setup and Usage

The requirements and usage instructions are similar to the previous script, with additional conditional logic to handle text based on its classification. Ensure you have your OpenAI API key set up for this enhanced functionality.

### Example

Run the script, and it will prompt for input. Depending on the input's classification, it may also provide a non-offensive version of the text:
```python
python script_name.py  # replace script_name.py with your actual Python script file name


In [ ]:
def preprocess_and_predict(text):
    # Clean the text
    cleaned_text = clean_text(text)

    # Tokenize the text
    encodings = tokenizer(cleaned_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Move tensors to the same device as model
    encodings = {key: val.to(device) for key, val in encodings.items()}

    # Evaluation mode
    model.eval()

    # Forward pass, no need to compute gradients
    with torch.no_grad():
        outputs = model(**encodings)

    # Get predictions
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predictions = torch.argmax(probabilities, dim=-1)

    # Convert predictions to labels
    label_map = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
    predicted_label = label_map[predictions.item()]

    # Get confidence scores
    confidence_scores = probabilities.squeeze().tolist()  # convert to list of probabilities

    # Check the label and process through OpenAI if necessary
    if predicted_label in ["Hate Speech", "Offensive Language"]:
        non_offensive_text = clean_speech(text)
        return predicted_label, confidence_scores, non_offensive_text
    else:
        return predicted_label, confidence_scores, None

def main():
    while True:
        user_input = input("Enter a tweet to analyze (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        predicted_label, confidence_scores, cleaned_text = preprocess_and_predict(user_input)
        print("Predicted label:", predicted_label)
        print("Confidence Scores:", confidence_scores)
        if cleaned_text:
            print("Cleaned Text:", cleaned_text)

# Run the main function
if __name__ == "__main__":
    main()


## Text Processing and Cleanup with OpenAI

This section of the research paper introduces two Python scripts designed to address the detection and modification of offensive or hate speech using machine learning and OpenAI's API.

### 1. Text Cleanup Function

This function uses OpenAI's GPT model to convert potentially offensive or hate speech into non-offensive language. It's particularly useful for moderating content or enhancing communication tools.

#### Code Snippet

```python
import openai

def clean_speech(input_text):
    try:
        # Use the OpenAI API to "translate" offensive speech to non-offensive speech
        response = openai.Completion.create(
          engine="text-davinci-003",  # You can choose a suitable model like davinci or curie
          prompt=f"Rewrite the following sentence to be polite and non-offensive:\n\n{input_text}",
          max_tokens=60  # Adjust max_tokens as necessary
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return str(e)

# Receive input from the user
user_input = input("Enter a potentially offensive sentence: ")
cleaned_text = clean_speech(user_input)
print("Cleaned Text:", cleaned_text)


### 2. Text Analysis and Conditional Cleanup

This script leverages a machine learning model to classify text into categories such as Hate Speech, Offensive Language, or Neither. If the text is determined to be offensive, it is automatically processed through the OpenAI API, which rewrites it into non-offensive language. This functionality is ideal for content moderation or enhancing communication tools in digital platforms.

#### Code Snippet

```python
def preprocess_and_predict(text):
    # Clean the text
    cleaned_text = clean_text(text)

    # Tokenize the text
    encodings = tokenizer(cleaned_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Move tensors to the same device as model
    encodings = {key: val.to(device) for key, val in encodings.items()}

    # Evaluation mode
    model.eval()

    # Forward pass, no need to compute gradients
    with torch.no_grad():
        outputs = model(**encodings)

    # Get predictions
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predictions = torch.argmax(probabilities, dim=-1)

    # Convert predictions to labels
    label_map = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
    predicted_label = label_map[predictions.item()]

    # Get confidence scores
    confidence_scores = probabilities.squeeze().tolist()

    # Check the label and process through OpenAI if necessary
    if predicted_label in ["Hate Speech", "Offensive Language"]:
        non_offensive_text = clean_speech(text)
        return predicted_label, confidence_scores, non_offensive_text
    else:
        return predicted_label, confidence_scores, None

def main():
    while True:
        user_input = input("Enter a tweet to analyze (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        predicted_label, confidence_scores, cleaned_text = preprocess_and_predict(user_input)
        print("Predicted label:", predicted_label)
        print("Confidence Scores:", confidence_scores)
        if cleaned_text:
            print("Cleaned Text:", cleaned_text)

if __name__ == "__main__":
    main()


## Setup and Usage

To utilize the text cleanup and analysis functionalities provided in these scripts, follow the instructions below to ensure a smooth setup and effective usage.

### Prerequisites

1. **Python Installation**: Ensure Python is installed on your machine. These scripts are tested with Python 3.8 and above.
2. **Library Installation**:
    - Install the `openai` library using pip to interact with the OpenAI API:
      ```bash
      pip install openai
      ```
    - If the second script requires PyTorch and other dependencies (like a specific NLP library for tokenization and model management), install them using:
      ```bash
      pip install torch transformers
      ```

3. **OpenAI API Key**:
    - Obtain an API key from OpenAI by registering on their platform. This key is essential for accessing the GPT model to clean offensive texts.
    - Configure the API key in your environment variables or directly within your script (ensure security best practices are followed if embedding directly in the script).

### Configuration

- Ensure the machine learning model and tokenizer are set up and configured properly for the second script:
  ```python
  from transformers import AutoModel, AutoTokenizer

  model = AutoModel.from_pretrained("bert-base-uncased")
  tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
